<a href="https://colab.research.google.com/github/ArtyomShabunin/SMOPA-25/blob/main/lesson_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://prana-system.com/files/110/rds_color_full.png" alt="tot image" width="300"  align="center"/> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src="https://mpei.ru/AboutUniverse/OficialInfo/Attributes/PublishingImages/logo1.jpg" alt="mpei image" width="200" align="center"/>
<img src="https://mpei.ru/Structure/Universe/tanpe/structure/tfhe/PublishingImages/tot.png" alt="tot image" width="100"  align="center"/>

---

# **Системы машинного обучения и предиктивной аналитики в тепловой и возобновляемой энергетике**  

# ***Практические занятия***


---

# Занятие №5
# Разметка данных для решения задачи многоклассовой классификации.
**19 марта 2025г.**

Задача **классификации** в машинном обучении — это тип задачи, в которой модель обучается определять, к какому из заранее заданных классов относится входной объект.  

**Принцип работы:**
1. **Входные данные (признаки)**: Набор характеристик объекта, представленных в виде вектора.
2. **Выходные данные (классы)**: Метка, указывающая, к какому классу принадлежит объект.
3. **Обучение модели**: На основе размеченного набора данных модель учится определять зависимости между признаками и классами.
4. **Предсказание**: На новых данных модель предсказывает класс объекта.

**Виды классификации:**
- **Бинарная** – два класса (например, "болен" или "здоров").
- **Многоклассовая** – более двух классов (например, категории товаров: "еда", "одежда", "электроника").
- **Многомарочная (multi-label)** – объект может относиться сразу к нескольким классам (например, изображение с метками "собака" и "улица").

**Связь с задачей детекции аномалий**
Методы детекции аномалий оценивают отклонение от нормального многообразия. Но такая оценка не дает информации о том, какие действия следует предпринять. Задача классификации позволяет определить к какому из заранее определенных классов аномалий отнести конкретную аномалию, что в свою очередь позволяет *быстро* принять адекватное решение.

In [ ]:
pip install --upgrade plotly-resampler

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme(rc={'figure.figsize':(15,5)})

from tqdm import tqdm
import glob
import json

import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go

from plotly_resampler import register_plotly_resampler, unregister_plotly_resampler

pd.options.mode.chained_assignment = None

import warnings
warnings.filterwarnings('ignore')

In [ ]:
register_plotly_resampler(mode="auto", default_n_shown_samples=10000)

In [ ]:
plot_layout = go.Layout(
    autosize=False,
    width=1000,
    height=400,
    # xaxis=go.layout.XAxis(linecolor="black", linewidth=1, mirror=True),
    # yaxis=go.layout.YAxis(linecolor="black", linewidth=1, mirror=True),
    margin=go.layout.Margin(l=50, r=50, b=10, t=40, pad=4),
    showlegend = True
)

Вспомогательные функции

In [ ]:
def plot_graph():

    params_dropdown = widgets.Dropdown(
        options=data.columns,
        description='Наименование сигнала',
        disabled=False,
        value=None
    )

    display(params_dropdown)
    out = widgets.Output()
    display(out)

    fig = go.Figure(layout=plot_layout)

    @out.capture()
    def params_dropdown_eventhandler(change):

        fig.add_traces(
            [
                {"x": data[change.new].index, "y": data[change.new].values, "name": change.new, "type": "scattergl"},
            ],
        )

        clear_output()
        display(fig)

    params_dropdown.observe(params_dropdown_eventhandler, names='value')

In [ ]:
def plot_graph_by_description():

    params_dropdown = widgets.Dropdown(
        options=description_to_kks.keys(),
        description='Описание сигнала',
        disabled=False,
        value=None
    )

    display(params_dropdown)
    out = widgets.Output()
    display(out)

    fig = go.Figure(layout=plot_layout)

    @out.capture()
    def params_dropdown_eventhandler(change):

        kks = description_to_kks[change.new]
        fig.add_traces(
            [
                {"x": data[kks].index, "y": data[kks].values, "name": kks, "type": "scattergl"},
            ],
        )

        clear_output()
        display(fig)

    params_dropdown.observe(params_dropdown_eventhandler, names='value')

In [ ]:
def plot_graph_with_modes(binary_features):
    # Виджет для выбора диапазона дат
    slider_layout = widgets.Layout(width='800px')

    date_range_slider = widgets.SelectionRangeSlider(
        options=[(date.strftime('%Y-%m-%d'), date) for date in data.index],
        index=(0, len(data.index) - 1),
        description='Временной диапазон',
        continuous_update=False,
        layout=slider_layout
    )

    # Виджет для выбора числового признака
    feature_selector = widgets.Dropdown(
        options=data.columns,
        value='GTA1.DBinPU.P',
        description='Сигнал:',
        style={'description_width': 'initial'}
    )

    # Виджет для выбора бинарных признаков
    binary_selector = widgets.SelectMultiple(
        options=binary_features,
        value=binary_features,  # По умолчанию выбраны все бинарные признаки
        description='Бинарные признаки:',
        style={'description_width': 'initial'}
    )

    # Функция для обновления графиков
    def update_plot(date_range, feature, binary_features):
        start, end = date_range

        # Создаем два графика, расположенных вертикально
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), height_ratios=[2, 1])
        mask = (data.index >= start) & (data.index <= end)

        # Верхний график: основной признак
        ax1.plot(
            data.index[mask], data[feature][mask],
            label=feature,
            ls='-', marker='.',
            # color='black',
            # linewidth=2
        )
        ax1.set_ylabel(feature)
        ax1.set_title(kks_to_description[feature])
        ax1.grid(True, axis='both')  # Добавляем сетку на верхний график (горизонтальные и вертикальные линии)
        ax1.set_xlim([start, end])
        # ax1.legend(loc='upper right')

        # Убираем метки оси X на верхнем графике
        # ax1.set_xticks([])
        ax1.set_xticklabels([])

        # Нижний график: бинарные признаки
        for binary_feature in binary_features:
            ax2.fill_between(data.index[mask], 0, 1, where=data[binary_feature][mask] == 1, alpha=0.7, label=binary_feature)
        ax2.set_ylabel('Режимы')
        ax2.set_yticks([0, 1])
        # ax2.set_title('Binary Features Over Time')
        ax2.grid(True, axis='x')  # Добавляем только вертикальные линии сетки на нижний график
        ax2.set_xlim([start, end])
        ax2.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=4)

        # Убираем расстояние между графиками
        plt.subplots_adjust(hspace=0)

        # Настройка общего вида
        # plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    # Виджет интерактивности
    display(widgets.VBox([
        date_range_slider,
        feature_selector,
        binary_selector,
        widgets.interactive_output(update_plot, {
            'date_range': date_range_slider,
            'feature': feature_selector,
            'binary_features': binary_selector
        })
    ]))

In [ ]:
def search_signal():
    search_input = widgets.Text(placeholder="Введите текст для поиска")
    output = widgets.Output()

    def on_text_change(change):
        with output:
            output.clear_output()
            query = change["new"].lower()
            filtered_items = [f'{item} - {description_to_kks[item]}' for item in description_to_kks.keys() if query in item.lower()]
            display(filtered_items if filtered_items else "Нет совпадений")

    search_input.observe(on_text_change, names='value')
    display(search_input, output)


### Загрузка данных

In [ ]:
# import gdown
# url = "https://drive.google.com/drive/folders/1RtrAevJUYSgTbp0YUztxEBB8_VcvjgGH?usp=drive_link"
# gdown.download_folder(url, quiet=True, verify=False)

In [ ]:
parquetFileList = glob.glob(f'./option_0/*.gzip')

In [ ]:
parquetFileList

Читаем все файлы и объединяем их в общий DataFrame

In [ ]:
df_list = []

for file in tqdm(parquetFileList):
    df = pd.read_parquet(file)
    df_list.append(df)

data = pd.concat(df_list, axis=0).sort_index().ffill().drop_duplicates()
data = data.dropna()

In [ ]:
print(f"Размерность датасета: {data.shape[0]} строк и {data.shape[1]} столбцов")

In [ ]:
data.head()

Чтение файла с описанием сигналов

In [ ]:
with open(f'./option_0/description.json', 'r', encoding = "utf-8") as f:
    description = json.load(f)

Составим словарь для трактовки наименований сигналов

In [ ]:
kks_to_description = {param['real_kks']: f"{param['description']}, [{param['unit']}]"
for param in description if param['real_kks'] in data.columns}

description_to_kks = { f"{param['description']}, [{param['unit']}]": param['real_kks']
for param in description if param['real_kks'] in data.columns}

### Разметка данных

Создание размеченных наборов данных с неисправностями энергетического оборудования представляет значительные трудности на практике. Эти сложности связаны с несколькими ключевыми факторами:  

1. **Редкость и однообразие аномальных данных**  
Отказы и неисправности в энергетическом оборудовании происходят относительно редко, особенно если применяется качественное техническое обслуживание. Это приводит к тому, что в исторических данных аномальные события встречаются крайне нечасто. Кроме того, даже когда такие случаи фиксируются, их вариативность может быть низкой — одни и те же типы отказов, что затрудняет обучение модели на разнообразных сценариях.  

2. **Ограниченное количество экспертов для разметки**  
Разметка данных требует глубокого понимания физических процессов и механизмов отказов. Только высококвалифицированные специалисты могут достоверно определять, является ли наблюдаемое изменение параметров работы оборудования признаком неисправности или результатом штатного изменения режима. Однако число таких экспертов ограничено, а их время дорого, что существенно замедляет процесс разметки.  

3. **Неоднозначность интерпретации данных**  
В реальных условиях многие аномальные ситуации развиваются постепенно, а их проявления могут пересекаться с нормальными колебаниями параметров работы оборудования. Это приводит к субъективности в разметке данных: один эксперт может классифицировать ситуацию как предаварийную, в то время как другой посчитает её допустимой.  

4. **Разнообразие оборудования и условий эксплуатации**  
Даже одинаковые типы оборудования могут работать в разных условиях — климатических, нагрузочных, эксплуатационных. Это создаёт дополнительные сложности при разметке, так как одна и та же неисправность может проявляться по-разному в зависимости от контекста.  

5. **Ограниченные архивные данные и их качество**  
Важные данные о неисправностях часто отсутствуют или хранятся в неудобных для анализа форматах (бумажные журналы, неструктурированные отчёты). Кроме того, в архивных данных могут быть пропуски, ошибки измерений или некорректные временные метки, что затрудняет их использование в обучении моделей.  

**Чтобы не упираться в ограничения связанные с отсутствием данных по неисправному состоянию агрегата, заменим задачу класификации дефектов на задачу класификации эксплуатационных режимов.**



#### **Классы режимов для ГТУ:**
* пусковой режим
* режим останова
* режим номинальной нагрузки
* режим пониженной нагрузки
* режим изменения нагрузки
* система подогрева воздуха включена
* остановленное состояние

### Классификация на основе правил

Задача классификации эксплуатационных режимов может быть решена как с помощью машинного обучения, так и с помощью экспертных правил.  

**Разработка математических моделей на основе экспертных знаний** – это методология, при которой модель строится не на основании больших массивов данных и машинного обучения, а на основе знаний экспертов и формализованных правил. Такой подход часто используется, когда:  

1. **Недостаточно данных** – например, аварийные или нештатные режимы редки, и обучить нейросеть на таких примерах сложно.  
2. **Высокие требования к интерпретируемости** – правила, основанные на экспертных знаниях, прозрачны и объяснимы, в отличие от "черного ящика" нейросетей.  
3. **Доменная специфика** – эксперты могут использовать накопленный опыт для явного кодирования физических и логических зависимостей.  

**Комбинированные подходы**  
На практике часто сочетают оба метода. Например:  
1. **Гибридная модель** – экспертные правила используются для предобработки данных и фильтрации аномалий, а затем применяется машинное обучение.  
2. **Использование правил для объяснения решений модели** – например, после классификации нейросетью можно наложить интерпретируемые правила для проверки корректности.

### Анализ и разметка данных

In [ ]:
from PIL import Image

image = Image.open(".\option_0\scheme.png")
display(image)

#### **Дадим определение эксплуатационным режимам которые будем классифицировать**

**Режим номинальной нагрузки**  
Работа ГТУ в зоне максимальной электрической нагрузки согласно имеющимся архивным данным.

**Режим пониженной нагрузки**  
Работа ГТУ в границах регулировочного диапазона, при нагрузки ниже номинальной.

**Режим увеличения / снижения нагрузки**  
Изменение электрической нагрузки в границах регулировочного диапазон.

**Режим пуска**  
Начало - увеличение оборотов выше скорости вращения валоповоротного устройства.  
Конец - увеличение электрической нагрузки выше нижней границы регулировочного диапазона.

**Режим останова**  
Начало - снижение электрической нагрузки ниже нижней границы регулировочного диапазона.  
Конец - снижение оборотов до скорости вращения валоповоротного устройства или нуля.

**Остановленное состояние**  
Обороты ниже скорости вращения валоповоротного устройства или отсутствует подача топлива и не наблюдается снижение оборотов (выбег ротора завершен).

**Режим подогрева воздуха**  
Генератор синхронизирован с сетью, температура воздуха перед ГТУ выше температуры наружного воздуха, открыта задвижка защиты от обледенения (ЗЗО).

#### **На основе каких сигналов можно определить класс эксплуатационного режима.**

**Режим номинальной нагрузки:**
- активная мощность генератора

**Режим пониженной нагрузки:**
- активная мощность генератора

**Режим увеличения / снижения нагрузки:**
- активная мощность генератора

**Режим пуска / останова:**
- активная мощность генератора
- скорость вращения ротора силовой турбины (СТ)
- скорость вращения ротора турбокомпрессора (ТК)
- расход топлива
- положение топливного клапана

**Режим подогрева воздуха:**
- температура наружного воздуха
- температура воздуха перед компрессором
- положение клапана подачи греющего воздуха

In [ ]:
plot_graph()

In [ ]:
plot_graph_by_description()

In [ ]:
search_signal()

#### Выделим дополнительные признаки

силовая турбина вращается с номинальной скоростью

In [ ]:
# data['full_speed'] = ...

генератор синхронизирован с сетью

In [ ]:
# data['gen_synch'] = ...

номинальная нагрузка

In [ ]:
# data['power_full'] = ...

среднее изменение мощности за 1 мин

In [ ]:
# data['diff_P'] = ...

увеличение мощности / снижение мощности

In [ ]:
# data['increase_power'] = ...
# data['decrease_power'] = ...

регулировочный диапазон электрической нагрузки

In [ ]:
# data['adjustment_range'] = ...

средняя скорость изменения оборотов силовой турбины

In [ ]:
# data['diff_nst'] = ...

увеличение / снижение скорости вращения силовой турбины

In [ ]:
# data['increase_nst'] = ...
# data['decrease_nst'] = ...

изменение скорости вращения турбокомпрессора

In [ ]:
# data['diff_ntk'] = ...

увеличение / снижение скорости вращения турбокомпрессора

In [ ]:
# data['increase_ntk'] = ...
# data['decrease_ntk'] = ...

нет подачи топлива

In [ ]:
# data['fuel_off'] = ...

разница между температурой воздуха перед ГТД и температурой наружного воздуха

In [ ]:
# data['diff_Tin'] = ...

наблюдается нагрев воздуха

In [ ]:
# data['heated_air'] = ...

ЗЗО открыт

In [ ]:
# data['zzo_is_open'] = ...

#### Классифицируем режимы

режим номинальной нагрузки

In [ ]:
data["full_power_mode"] = False

режим не полной нагрузки

In [ ]:
data["partial_power_mode"] = False

режим увеличения нагрузки

In [ ]:
data["increas_power_mode"] = False

режим снижения нагрузки

In [ ]:
data["decreas_power_mode"] = False

режим пуска

In [ ]:
data["start_up_mode"] = False

режим останова

In [ ]:
data["shutdown_mode"] = False

остановленное состояние

In [ ]:
data["stopped_state_mode"] = False

режим подогрева воздуха

In [ ]:
data["air_heating_mode"] = False

### Визуализация разметки

In [ ]:
plot_graph_with_modes([
    "full_power_mode", "partial_power_mode", "increas_power_mode", "decreas_power_mode",
    "start_up_mode", "shutdown_mode", "stopped_state_mode", "air_heating_mode"])

### Сохранение подготовленных данных

In [ ]:
df.to_parquet("data_modes.gzip", compression='gzip')